In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load Data
train_df = pd.read_csv("../input/playground-series-s4e5/train.csv")
test_df = pd.read_csv("../input/playground-series-s4e5/test.csv")

In [4]:
# test_df.head()

In [5]:
# print(train_df.head())
# display(train_df.columns)

In [6]:
# train_df.dtypes

In [7]:
# train_df.isna().any()

In [8]:
# plt.figure(figsize=(20, 20))
# sns.heatmap(train_df.corr(), vmax=1, vmin=-1, cmap='RdBu', annot=True)

In [9]:
# train_df.hist(figsize=(20, 20), xrot=-45)
# plt.show()

In [10]:
# train_df.nunique()

In [11]:
# train_df.shape

Notes:

1. There are no missing values

2. The data is made of 21 categorical columns that are already encoded as integers and a label column in float

3. The distribution of the columns is a guassian distribution

4. There's no significant correlation between the features.

In [12]:
X = train_df.drop(['FloodProbability', 'id'], axis = 1)
y = train_df['FloodProbability']

In [13]:
# Split data into training and test sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# # Mutual Information

# from sklearn.feature_selection import mutual_info_regression

# def make_mi_scores(X, y):
#     mi_scores = mutual_info_regression(X, y)
#     mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
#     return mi_scores

# def plot_mi_scores(scores):
#     scores = scores.sort_values(ascending=True)
#     width = np.arange(len(scores))
#     ticks = list(scores.index)
#     plt.barh(width, scores)
#     plt.yticks(width, ticks)
#     plt.title("Mutual Information Scores")

In [15]:
# mi_scores = make_mi_scores(X, y)

In [16]:
# plt.figure(dpi=100, figsize=(8, 5))
# plot_mi_scores(mi_scores)

In [17]:
from sklearn.model_selection import cross_val_score

def score_dataset(X, y, model=XGBRegressor(n_jobs=4, verbosity=2, random_state=1)):
    
    score = cross_val_score(
        model, X, y, cv=5, scoring="r2",
    )
    score = score.mean()
    score = np.sqrt(score)
    return score

In [18]:
def score_valid(X_train, y_train, X_valid, y_valid):
    model=XGBRegressor(n_jobs=4, verbosity=2, random_state=1, early_stopping_rounds = 5)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose = False)
    y_pred = model.predict(X_valid)
    r2 = r2_score(y_valid, y_pred)
    return r2

In [19]:
# score_dataset(X, y)

In [20]:
# score_valid(X_train, y_train, X_valid, y_valid)

In [21]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(degree=2)
# X_poly_train = poly.fit_transform(X)
# X_t, X_v, y_t, y_v = train_test_split(X_poly_train, y, test_size=0.2, random_state=42)
# score_valid(X_t, y_t, X_v, y_v)


In [22]:
# score_dataset(X_poly_train, y)

In [23]:
df = train_df.copy()
df = df.drop(['FloodProbability'],axis = 1)

In [24]:
def generate_features(dataset):
    features = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement',
       'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality',
       'Siltation', 'AgriculturalPractices', 'Encroachments',
       'IneffectiveDisasterPreparedness', 'DrainageSystems',
       'CoastalVulnerability', 'Landslides', 'Watersheds',
       'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss',
       'InadequatePlanning', 'PoliticalFactors']
    
    
    dataset['mean_features'] = 0.1*dataset[features].mean(axis=1)
    dataset['std_features'] = dataset[features].std(axis=1)
    dataset['max_features'] = dataset[features].max(axis=1)
    dataset['min_features'] = dataset[features].min(axis=1)
    dataset['median_features'] = 0.1*dataset[features].median(axis=1)
    dataset['range_features'] = dataset['max_features'] - dataset['min_features']
    dataset['variance_features'] = dataset[features].var(axis=1)
    dataset['skewness_features'] = dataset[features].skew(axis=1)
    
    quartiles = dataset[features].quantile([0.25, 0.5, 0.75], axis=1)
    dataset['first_quartile'] = quartiles.loc[0.25]
    dataset['second_quartile'] = quartiles.loc[0.5]
    dataset['third_quartile'] = quartiles.loc[0.75]
    
    kurtosis_features = dataset[features].kurtosis(axis=1)
    dataset['kurtosis_features'] = kurtosis_features
    
    selected_columns = ['Urbanization', 'Deforestation', 'AgriculturalPractices', 'InadequatePlanning']
    dataset['Sum_Selected_Columns'] = dataset[selected_columns].sum(axis=1)
    
    dataset['Urbanization_Deforestation_Interaction'] = dataset['Urbanization'] * dataset['Deforestation']
    dataset['Urbanization_TopographyDrainage_Ratio'] = dataset['Urbanization'] / (dataset['TopographyDrainage'] + 1)
    
    dataset.drop(features, axis=1, inplace=True)
    
    return dataset

In [25]:
new_df = generate_features(df)

In [26]:
t_df = test_df.copy()
new_test = generate_features(t_df)

In [27]:
X_t2, X_v2, y_t2, y_v2 = train_test_split(new_df, y, test_size=0.2, random_state=42)

In [28]:
# score_valid(X_t2, y_t2, X_v2, y_v2)

In [29]:
# score_dataset(new_df, y)

# XGBoost Grid Search

In [30]:
# # Initialize XGBoost regressor
# xgb_reg = XGBRegressor(n_jobs=4, random_state=1, early_stopping_rounds = 5, verbosity=2)

# # Define hyperparameters grid for Grid Search
# param_grid = {
#     'alpha': [0, 0.1, 0.5, 1, 2], 
#     'lambda': [0, 0.1, 0.5, 1, 2],  
#     'learning_rate': [0.001, 0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'n_estimators': [50, 100, 200, 400]
# }

# # Initialize Grid Search with cross-validation
# grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, cv=3, scoring='r2', verbose = 1)

# # Perform Grid Search to find the best hyperparameters
# fit_params = {
#     'eval_set': [(X_valid, y_valid)],
#     'verbose': True
# }
# grid_search.fit(X_train, y_train, **fit_params)

In [31]:
# # Get the best hyperparameters
# best_params = grid_search.best_params_

# # Train XGBoost regressor with the best hyperparameters
# best_xgb_reg = XGBRegressor(n_jobs=4, random_state=1, early_stopping_rounds = 5, verbosity=2, **best_params)
# best_xgb_reg.fit(X_train, y_train, **fit_params)

# # Predict on the test set
# y_pred = best_xgb_reg.predict(X_valid)

# # Calculate R² score on the test set
# r2 = r2_score(y_valid, y_pred)
# print("R² Score:", r2)

In [32]:
# # best grid search param

# # Get the best estimator and evaluate on the validation set
# best_grid_reg = grid_search.best_estimator_
# y_pred_2 = best_grid_reg.predict(X_valid)

# # Calculate R² score
# from sklearn.metrics import r2_score
# r2 = r2_score(y_valid, y_pred_2)
# print("R² Score on Validation Set:", r2)


# Submission

In [33]:
#Sample model
model=XGBRegressor(n_jobs=4, verbosity=2, random_state=1)
model.fit(new_df, y, verbose = False)
y_pred = model.predict(new_test)

In [34]:
sub = pd.read_csv('/kaggle/input/playground-series-s4e5/sample_submission.csv')

In [35]:
sub['FloodProbability'] = y_pred

In [36]:
sub.to_csv("submission.csv", index=False)